# Dependency setup

## Package setup

In [175]:
# Import dependencies and libraries
from IPython.display import clear_output
import pandas as pd
from census import Census
from us import states
from dotenv import load_dotenv
import requests
import sys
import csv
import os
import json
import time

# Load development mode
try:
  dev = os.getenv("DEV_MODE")
  if dev: print("Development mode loaded successfully!")
except:
  print("Error loading development mode")
  print("Defaulting to production mode")
  dev = False

# Load environment variables and U.S. Census API key
if dev: print("Loading environment variables...")
load_dotenv()
if dev: print("Environment variables loaded successfully!")
try:
  if dev: print("Loading U.S. Census API key...")
  api_key = os.getenv("CENSUS_API_KEY")
  if dev: print("API key loaded successfully!")
except:
  print("Error loading U.S. Census API key")
  print("Please check your environment variables")
  

Development mode loaded successfully!
Loading environment variables...
Environment variables loaded successfully!
Loading U.S. Census API key...
API key loaded successfully!


## Google Colab setup

In [ ]:
# Install the required libraries for running on colab
try:
  %pip install prophet
except:
  print("Error installing libraries to colab instance")
finally:
  clear_output()
  print("Libraries successfully installed to colab instance")


# Census Code

In [176]:
# Create an instance of the Census library
try:
  if dev: print("Creating an instance of the Census API...")
  c = Census(
      api_key
  )
  if dev: print("Census API instance created:")
  if dev: display(type(c))
  if dev: display(c)
except:
  print("Error creating an instance of the Census API")
  print("Please check your U.S. Census API key")


Creating an instance of the Census API...
Census API instance created:


census.core.Census

## Census API Aliases

In [177]:
# Load census aliases
# TODO: Add google colab-specific code to load the census aliases
if dev: print("Loading census aliases...")
with open('data/census/census_aliases.json') as f:
    census_aliases = json.load(f)

# Display the census aliases
# display(census_aliases)

def printAliases():
  # Show the keys in the census aliases
  alias_count = 1
  print("Census aliases:")
  print("-"*30)
  for key, value in census_aliases.items():
      description = value["description"]
      column_name = value["variable"]
      title = value["text"]
      unit = value["unit"]
      print(f"{alias_count:03d} | {title.capitalize()}: {description}")
      print(f"    | Census column name: {column_name}")
      print(f"    | Unit: {unit}")
      print("-"*30)
      alias_count+=1

# Saving the reference of the standard output
original_stdout = sys.stdout 	

try:
  # Save the census aliases to a file
  if dev: print("Saving census aliases to file...")
  with open('census_aliases.txt', 'w') as f:
      sys.stdout = f
      printAliases()
      # Reset the standard output
      sys.stdout = original_stdout
  print("Census aliases saved to file successfully!")
except:
  print("Error saving census aliases to file")


Loading census aliases...
Saving census aliases to file...
Census aliases saved to file successfully!


## FIPS Codes

### Functions for obtaining county FIPS codes

In [178]:
# Define funtions to get FIPS codes for US counties
# https://gist.github.com/cjwinchester/a8ff5dee9c07d161bdf4
def getCounties():
    "Function to return a dict of FIPS codes (keys) of U.S. counties (values)"
    d = {}
    r = requests.get("http://www2.census.gov/geo/docs/reference/codes/files/national_county.txt")
    reader = csv.reader(r.text.splitlines(), delimiter=',')    
    for line in reader:
        d[line[1] + line[2]] = line[3].replace(" County","")    
    return d

def getCountyAdj():
    "Return a list of dicts where each dict has a county FIPS code (key) and a list of FIPS codes of the adjacent counties, not including that county (value)"
    adj = requests.get("http://www2.census.gov/geo/docs/reference/county_adjacency.txt")
    adj_data = adj.text.encode("utf-8")
    reader = csv.reader(adj_data.splitlines(), delimiter='\t')
    ls = []
    d = {}
    countyfips = ""
    for row in reader:
        if row[1] and row[1] != "":
            if d:
                ls.append(d)
            d = {}
            countyfips = row[1]
            d[countyfips] = []
            "Grab the record on the same line"
            try:
                st = row[3]
                if st != countyfips:
                    d[countyfips].append(st)
            except:
                pass
        else:
            "Grab the rest of the records"
            if row[3] and row[3] != "":
                st = row[3]
                if st != countyfips:
                    d[countyfips].append(st)


### Obtaining state FIPS codes

In [179]:
# Get the FIPS codes for Colorado
try:
  if dev: print("Getting FIPS codes for Colorado...")
  colorado_fips = states.CO.fips
  state_fips = colorado_fips
  print(f"Colorado FIPS code: {state_fips}")
except:
  print("Error getting FIPS codes for Colorado")


Getting FIPS codes for Colorado...
Colorado FIPS code: 08


### Obtaining all US county FIPS codes

In [180]:
# Store county FIPS codes in memory
try:
  if dev: print("Storing county FIPS codes in memory...")
  county_fips = getCounties()
  if dev: print("All County FIPS codes:")
  if dev: display(county_fips)
except:
  print("Error storing county FIPS codes in memory")
  

Storing county FIPS codes in memory...
All County FIPS codes:


{'01001': 'Autauga',
 '01003': 'Baldwin',
 '01005': 'Barbour',
 '01007': 'Bibb',
 '01009': 'Blount',
 '01011': 'Bullock',
 '01013': 'Butler',
 '01015': 'Calhoun',
 '01017': 'Chambers',
 '01019': 'Cherokee',
 '01021': 'Chilton',
 '01023': 'Choctaw',
 '01025': 'Clarke',
 '01027': 'Clay',
 '01029': 'Cleburne',
 '01031': 'Coffee',
 '01033': 'Colbert',
 '01035': 'Conecuh',
 '01037': 'Coosa',
 '01039': 'Covington',
 '01041': 'Crenshaw',
 '01043': 'Cullman',
 '01045': 'Dale',
 '01047': 'Dallas',
 '01049': 'DeKalb',
 '01051': 'Elmore',
 '01053': 'Escambia',
 '01055': 'Etowah',
 '01057': 'Fayette',
 '01059': 'Franklin',
 '01061': 'Geneva',
 '01063': 'Greene',
 '01065': 'Hale',
 '01067': 'Henry',
 '01069': 'Houston',
 '01071': 'Jackson',
 '01073': 'Jefferson',
 '01075': 'Lamar',
 '01077': 'Lauderdale',
 '01079': 'Lawrence',
 '01081': 'Lee',
 '01083': 'Limestone',
 '01085': 'Lowndes',
 '01087': 'Macon',
 '01089': 'Madison',
 '01091': 'Marengo',
 '01093': 'Marion',
 '01095': 'Marshall',
 '01097': 

### Obtaining FIPS codes for all counties in Colorado

In [181]:
# Filter out only Colorado county FIPS codes
try:
  if dev: print("Filtering out only Colorado county FIPS codes...")
  colorado_county_fips = {key: value for key, value in county_fips.items() if key.startswith(state_fips)}
  if dev: print("Colorado County FIPS codes:")
  if dev: display(colorado_county_fips)
  print(f"Number of Colorado counties: {len(colorado_county_fips)}")
except:
  print("Error filtering out only Colorado county FIPS codes")


Filtering out only Colorado county FIPS codes...
Colorado County FIPS codes:


{'08001': 'Adams',
 '08003': 'Alamosa',
 '08005': 'Arapahoe',
 '08007': 'Archuleta',
 '08009': 'Baca',
 '08011': 'Bent',
 '08013': 'Boulder',
 '08014': 'Broomfield',
 '08015': 'Chaffee',
 '08017': 'Cheyenne',
 '08019': 'Clear Creek',
 '08021': 'Conejos',
 '08023': 'Costilla',
 '08025': 'Crowley',
 '08027': 'Custer',
 '08029': 'Delta',
 '08031': 'Denver',
 '08033': 'Dolores',
 '08035': 'Douglas',
 '08037': 'Eagle',
 '08039': 'Elbert',
 '08041': 'El Paso',
 '08043': 'Fremont',
 '08045': 'Garfield',
 '08047': 'Gilpin',
 '08049': 'Grand',
 '08051': 'Gunnison',
 '08053': 'Hinsdale',
 '08055': 'Huerfano',
 '08057': 'Jackson',
 '08059': 'Jefferson',
 '08061': 'Kiowa',
 '08063': 'Kit Carson',
 '08065': 'Lake',
 '08067': 'La Plata',
 '08069': 'Larimer',
 '08071': 'Las Animas',
 '08073': 'Lincoln',
 '08075': 'Logan',
 '08077': 'Mesa',
 '08079': 'Mineral',
 '08081': 'Moffat',
 '08083': 'Montezuma',
 '08085': 'Montrose',
 '08087': 'Morgan',
 '08089': 'Otero',
 '08091': 'Ouray',
 '08093': 'Park',
 

Number of Colorado counties: 64


### Obtaining FIPS code for a specific county in Colorado

In [184]:
# Prompt for county to look up and retrieve fip
# county_name = input("Enter county name to look up the FIPS code: ")
try:
  if dev: print("Looking up the FIPS code for Denver county...")
  county_name = "Denver"
  state_county_fip = list(county_fips.keys()) [list(county_fips.values()).index(county_name)]
  county_fip = state_county_fip[2:]
  print(f"The FIPS code for {county_name} is {county_fip}")
except:
  print(f"Error looking up the FIPS code for {county_name} county")


Looking up the FIPS code for Denver county...
The FIPS code for Denver is 031


## Census Data

### Getting Raw Census Data

In [193]:
# Run Census Search to retrieve data on all zip codes
census_start_year = 2011
census_end_year = 2021
census_years = range(census_start_year, census_end_year + 1)
data_retrieval_output = sys.stdout

# Create a DataFrame to hold the census data with an empty row so we can append data to it
raw_census_df = pd.DataFrame({
  "NAME": "",
  "B19013_001E": "",
  "B01003_001E": "",
  "B01002_001E": "",
  "B19301_001E": "",
  "B17001_002E": "",
  "B17012_003E": "",
  "Year": ""
}, index=[0])

# Loop through the years and retrieve data for each year
try:
  for county_fip in colorado_county_fips.keys():
    for year in census_years:
      if dev: print(f"Retrieving data for {year} {colorado_county_fips[county_fip]} County...")
      year_data = c.acs5.state_county(
        (
            "NAME",
            "B19013_001E",
            "B01003_001E",
            "B01002_001E",
            "B19301_001E",
            "B17001_002E",
            "B17012_003E"
        ),
        colorado_fips,
        county_fip[2:],
        year=year
      )

      # Display the retrieved data
      if dev: print(f"Data retrieved for {year} {colorado_county_fips[county_fip]} County.")
      if dev: display(year_data)

      # Convert to DataFrame
      if dev: print("Converting data to DataFrame...")
      year_df = pd.DataFrame(year_data)

      # Add the data's year to a Year column
      if dev: print(f"Adding Year column to {year} {colorado_county_fips[county_fip]} County...")
      year_df["Year"] = year

      # Add the data to the combined DataFrame
      if dev: print("Adding data to the combined DataFrame...")
      raw_census_df = pd.concat([raw_census_df, year_df], ignore_index=True)

      # Wait for 5 seconds before making the next request
      if dev: print("Waiting for 5 seconds before making the next request...")
      if dev: print("-"*50)
      # TODO: Show percentage of data retrieval completion
      clear_output(wait=True)
      time.sleep(5)
    print(f"Data retrieval of {colorado_county_fips[county_fip]} County complete.")
  print(f"Data retrieval of all Colorado counties complete.")
except Exception as e:
  print("Error retrieving data from the U.S. Census API")
  print(e)


Data retrieval complete.


### Saving Raw Census Data to CSV

In [194]:
# Save the raw census data to a CSV file
try:
  if dev: print("Saving raw census data to a CSV file...")
  raw_census_df.to_csv("data/census/raw_census_data.csv", index=False)
  print("Raw census data saved to file successfully!")
except:
  print("Error saving raw census data to file")
  

Saving raw census data to a CSV file...
Raw census data saved to file successfully!


### Loading Raw Census Data from CSV

In [195]:
# Load the raw census data from the CSV file
try:
  if dev: print("Loading raw census data from the CSV file...")
  raw_census_df = pd.read_csv("data/census/raw_census_data.csv")
  if dev: display(raw_census_df)
  print("Raw census data loaded from file successfully!")
except:
  print("Error loading raw census data from file, please make sure the file exists.")


Loading raw census data from the CSV file...


,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B17012_003E,Year,state,county
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Adams County, Colorado",56089.0,434295.0,32.2,24384.0,60147.0,5143.0,2011.0,8.0,1.0
2,"Adams County, Colorado",56633.0,442996.0,32.4,24357.0,62008.0,4999.0,2012.0,8.0,1.0
3,"Adams County, Colorado",56270.0,452030.0,32.6,24195.0,63540.0,4814.0,2013.0,8.0,1.0
4,"Adams County, Colorado",57421.0,461558.0,32.8,24667.0,64599.0,4861.0,2014.0,8.0,1.0
...,...,...,...,...,...,...,...,...,...,...
700,"Yuma County, Colorado",44668.0,10109.0,35.8,24789.0,1425.0,181.0,2017.0,8.0,125.0
701,"Yuma County, Colorado",48394.0,10069.0,38.4,25846.0,1438.0,202.0,2018.0,8.0,125.0
702,"Yuma County, Colorado",52022.0,10003.0,37.8,27955.0,1357.0,177.0,2019.0,8.0,125.0
703,"Yuma County, Colorado",54393.0,10013.0,38.9,28791.0,1099.0,190.0,2020.0,8.0,125.0


Raw census data loaded from file successfully!


### Cleaning Census Data

#### Dropping rows with missing values

In [196]:
# Drop the empty row from the combined DataFrame
try:
  if dev: print("Dropping the empty row from the combined DataFrame...")
  raw_census_df.dropna(inplace=True)
  if dev: print("Combined DataFrame:")
  display(raw_census_df)
except:
  print("Error dropping empty row from the combined DataFrame")


Dropping the empty row from the combined DataFrame...
Combined DataFrame:


,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B17012_003E,Year,state,county
1,"Adams County, Colorado",56089.0,434295.0,32.2,24384.0,60147.0,5143.0,2011.0,8.0,1.0
2,"Adams County, Colorado",56633.0,442996.0,32.4,24357.0,62008.0,4999.0,2012.0,8.0,1.0
3,"Adams County, Colorado",56270.0,452030.0,32.6,24195.0,63540.0,4814.0,2013.0,8.0,1.0
4,"Adams County, Colorado",57421.0,461558.0,32.8,24667.0,64599.0,4861.0,2014.0,8.0,1.0
5,"Adams County, Colorado",58946.0,471206.0,33.0,25039.0,64241.0,4802.0,2015.0,8.0,1.0
...,...,...,...,...,...,...,...,...,...,...
700,"Yuma County, Colorado",44668.0,10109.0,35.8,24789.0,1425.0,181.0,2017.0,8.0,125.0
701,"Yuma County, Colorado",48394.0,10069.0,38.4,25846.0,1438.0,202.0,2018.0,8.0,125.0
702,"Yuma County, Colorado",52022.0,10003.0,37.8,27955.0,1357.0,177.0,2019.0,8.0,125.0
703,"Yuma County, Colorado",54393.0,10013.0,38.9,28791.0,1099.0,190.0,2020.0,8.0,125.0


#### Index fixing and Sorting

In [206]:
# Copy the DataFrame to a new DataFrame
if dev: print("Copying the DataFrame to a new DataFrame...")
census_df = raw_census_df.copy()
# Reset the index of the DataFrame
if dev: print("Resetting the index of the DataFrame...")
census_df.reset_index(inplace=True, drop=True)
# Sort the DataFrame by Name and Year
if dev: print("Sorting the DataFrame by Name and Year...")
census_df.sort_values(by=["NAME", "Year"], inplace=True)
# Display the DataFrame
if dev: print("New DataFrame:")
if dev: display(census_df)


Copying the DataFrame to a new DataFrame...
Resetting the index of the DataFrame...
Sorting the DataFrame by Name and Year...
New DataFrame:


,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B17012_003E,Year,state,county
0,"Adams County, Colorado",56089.0,434295.0,32.2,24384.0,60147.0,5143.0,2011.0,8.0,1.0
1,"Adams County, Colorado",56633.0,442996.0,32.4,24357.0,62008.0,4999.0,2012.0,8.0,1.0
2,"Adams County, Colorado",56270.0,452030.0,32.6,24195.0,63540.0,4814.0,2013.0,8.0,1.0
3,"Adams County, Colorado",57421.0,461558.0,32.8,24667.0,64599.0,4861.0,2014.0,8.0,1.0
4,"Adams County, Colorado",58946.0,471206.0,33.0,25039.0,64241.0,4802.0,2015.0,8.0,1.0
...,...,...,...,...,...,...,...,...,...,...
699,"Yuma County, Colorado",44668.0,10109.0,35.8,24789.0,1425.0,181.0,2017.0,8.0,125.0
700,"Yuma County, Colorado",48394.0,10069.0,38.4,25846.0,1438.0,202.0,2018.0,8.0,125.0
701,"Yuma County, Colorado",52022.0,10003.0,37.8,27955.0,1357.0,177.0,2019.0,8.0,125.0
702,"Yuma County, Colorado",54393.0,10013.0,38.9,28791.0,1099.0,190.0,2020.0,8.0,125.0


#### Column fixing

In [207]:
# Convert the Year column to an integer
try:
  if dev: print("Converting the Year column to an integer...")
  census_df["Year"] = census_df["Year"].astype(int)
  if dev: print("Year column converted to an integer successfully!")
except:
  print("Error converting the Year column to an integer")


Converting the Year column to an integer...
Year column converted to an integer successfully!


In [208]:
# Column renaming
if dev: print("Renaming columns...")
census_df.rename(
  columns = {
    "NAME": "County",
    "B01003_001E": "Population",
    "B01002_001E": "Median Age",
    "B19013_001E": "Household Income",
    "B19301_001E": "Per Capita Income",
    "B17001_002E": "Poverty Count",
    "B17012_003E": "Families in Poverty (12mo)",
  },
  inplace=True
)

# Add a Poverty Rate column (Poverty Count / Population)
if dev: print("Adding a Poverty Rate column...")
census_df["Poverty Rate"] = 100 * census_df["Poverty Count"].astype(int) / census_df["Population"].astype(int)


Renaming columns...
Adding a Poverty Rate column...


In [214]:
# Configure the final DataFrame
if dev: print("Configuring the DataFrame...")
census_df = census_df[
  [
    "County",
    "Year",
    "Population",
    "Median Age",
    "Household Income",
    "Per Capita Income",
    "Poverty Count",
    "Poverty Rate",
    "Families in Poverty (12mo)"
  ]
]

# Display DataFrame length and sample data
if dev: print(f"Number of rows in the DataFrame: {len(census_df)}")
if dev: print("Combined county DataFrame preview:")
if dev: display(census_df.head())


Configuring the DataFrame...
Number of rows in the DataFrame: 704
Combined county DataFrame preview:


,County,Year,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Families in Poverty (12mo)
0,"Adams County, Colorado",2011,434295.0,32.2,56089.0,24384.0,60147.0,13.849342,5143.0
1,"Adams County, Colorado",2012,442996.0,32.4,56633.0,24357.0,62008.0,13.997418,4999.0
2,"Adams County, Colorado",2013,452030.0,32.6,56270.0,24195.0,63540.0,14.056589,4814.0
3,"Adams County, Colorado",2014,461558.0,32.8,57421.0,24667.0,64599.0,13.995858,4861.0
4,"Adams County, Colorado",2015,471206.0,33.0,58946.0,25039.0,64241.0,13.633315,4802.0


#### Saving Cleaned Census Data to CSV

In [210]:
# Save the final census data to a CSV file
try:
  if dev: print("Saving final census data to a CSV file...")
  census_df.to_csv("data/census/clean_census_data.csv", index=False)
  print("Final census data saved to file successfully!")
except:
  print("Error saving final census data to file")


Saving final census data to a CSV file...
Final census data saved to file successfully!


#### Loading Cleaned Census Data from CSV

In [215]:
# Load the final census data from the CSV file
try:
  if dev: print("Loading final census data from the CSV file...")
  census_df = pd.read_csv("data/census/clean_census_data.csv")
  if dev: display(census_df.head())
  print("Final census data loaded from file successfully!")
except:
  print("Error loading final census data from file, please make sure the file exists.")


Loading final census data from the CSV file...


,County,Year,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Families in Poverty (12mo)
0,"Adams County, Colorado",2011,434295.0,32.2,56089.0,24384.0,60147.0,13.849342,5143.0
1,"Adams County, Colorado",2012,442996.0,32.4,56633.0,24357.0,62008.0,13.997418,4999.0
2,"Adams County, Colorado",2013,452030.0,32.6,56270.0,24195.0,63540.0,14.056589,4814.0
3,"Adams County, Colorado",2014,461558.0,32.8,57421.0,24667.0,64599.0,13.995858,4861.0
4,"Adams County, Colorado",2015,471206.0,33.0,58946.0,25039.0,64241.0,13.633315,4802.0


Final census data loaded from file successfully!


In [212]:
# Display the first and last 30 rows of the DataFrame
print("First 30 rows of the DataFrame:")
display(census_df.head(30))

print("Last 30 rows of the DataFrame:")
display(census_df.tail(30))


,County,Year,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Families in Poverty (12mo)
0,"Adams County, Colorado",2011,434295.0,32.2,56089.0,24384.0,60147.0,13.849342,5143.0
1,"Adams County, Colorado",2012,442996.0,32.4,56633.0,24357.0,62008.0,13.997418,4999.0
2,"Adams County, Colorado",2013,452030.0,32.6,56270.0,24195.0,63540.0,14.056589,4814.0
3,"Adams County, Colorado",2014,461558.0,32.8,57421.0,24667.0,64599.0,13.995858,4861.0
4,"Adams County, Colorado",2015,471206.0,33.0,58946.0,25039.0,64241.0,13.633315,4802.0
5,"Adams County, Colorado",2016,479977.0,33.3,61444.0,26051.0,61265.0,12.764153,4690.0
6,"Adams County, Colorado",2017,487850.0,33.4,64087.0,27487.0,59050.0,12.104130,4542.0
7,"Adams County, Colorado",2018,497115.0,33.6,67575.0,28897.0,56588.0,11.383282,4181.0
8,"Adams County, Colorado",2019,504108.0,33.8,71202.0,30313.0,54159.0,10.743531,4100.0
9,"Adams County, Colorado",2020,509844.0,34.0,73817.0,31310.0,49946.0,9.796330,3647.0


,County,Year,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Families in Poverty (12mo)
674,"Washington County, Colorado",2014,4781.0,44.1,44271.0,24326.0,580.0,12.131353,83.0
675,"Washington County, Colorado",2015,4795.0,44.4,45541.0,24817.0,466.0,9.718457,74.0
676,"Washington County, Colorado",2016,4814.0,42.8,46315.0,25744.0,529.0,10.988783,75.0
677,"Washington County, Colorado",2017,4809.0,42.2,51458.0,26279.0,443.0,9.211894,71.0
678,"Washington County, Colorado",2018,4840.0,41.6,49375.0,26680.0,480.0,9.917355,58.0
679,"Washington County, Colorado",2019,4868.0,41.6,50094.0,27860.0,535.0,10.990140,68.0
680,"Washington County, Colorado",2020,4869.0,41.3,51181.0,28224.0,559.0,11.480797,57.0
681,"Washington County, Colorado",2021,4834.0,41.0,54141.0,29814.0,520.0,10.757137,54.0
682,"Weld County, Colorado",2011,248441.0,32.9,55825.0,25233.0,33351.0,13.424113,2822.0
683,"Weld County, Colorado",2012,253552.0,33.2,56589.0,25341.0,35581.0,14.033019,2758.0
